In [2]:
!pip install -U langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [8]:
# ================== INSTALL (run once if needed) ==================
# !pip install -U langchain langchain-google-genai google-genai

# ================== IMPORTS ==================
import os
import re
import json
import sqlite3

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ================== API KEY ==================
gemini_api_key = userdata.get("key_1")
if not gemini_api_key:
    raise ValueError("GOOGLE_API_KEY not found in Colab userdata. Set it first.")

# ================== DATABASE SETUP ==================
def setupdb():
    # timeout helps avoid "database is locked" errors
    with sqlite3.connect("example.db", timeout=10) as conn:
        cursor = conn.cursor()

        # Optional: better concurrency mode
        cursor.execute("PRAGMA journal_mode=WAL;")

        # Create table if not exists
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY,
            name TEXT,
            email TEXT,
            signup_date DATE
        )
        """)

        # Check if data already exists
        cursor.execute("SELECT COUNT(*) FROM customers")
        count = cursor.fetchone()[0]

        if count == 0:
            cursor.executemany("""
            INSERT INTO customers (id, name, email, signup_date)
            VALUES (?, ?, ?, ?)
            """, [
                (1, "Alice Johnson", "alice@example.com", "2025-06-15"),
                (2, "Bob Smith", "bob@example.com", "2024-07-01"),
                (3, "Clara White", "clara@example.com", "2024-07-10")
            ])
            print("✅ Sample data inserted.")
        else:
            print("ℹ️ Data already exists, skipping insert.")

        conn.commit()

# ================== LANGCHAIN HELPER ==================
def create_roles_chain(model):
    prompt = ChatPromptTemplate.from_template("{input}")
    return prompt | model | StrOutputParser()  # prompt -> model -> parser

# ================== SQL GENERATION WITH GEMINI ==================
def generate_sql(question: str) -> str:
    schema_description = """
    Table: customers
    - id (integer)
    - name (text)
    - email (text)
    - signup_date (date)
    """

    final_prompt = f"""
    ### You are an expert SQL query writer. You translate user requirements into valid SQLite SQL queries.
    Use the following database schema to generate the SQL.

    ### Database Schema:
    {schema_description}

    ### Question:
    {question}

    ### Rules:
    - Only use the given table and columns.
    - Do NOT explain anything.
    - Do NOT add comments.
    - Do NOT use backticks around table or column names.
    - The SQL must be directly executable in SQLite.

    ### Output format:
    Return the answer as a JSON object inside a ```json code block, like this:

    ```json
    {{
      "SQL": "SELECT * FROM customers;"
    }}
    ```
    """

    gemini_model = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite-001",   # you can also use "gemini-1.5-flash"
        google_api_key=gemini_api_key,
        temperature=0.3
    )

    print("\n--- LLM RAW RESPONSE ---\n")
    sql_response = create_roles_chain(gemini_model).invoke({"input": final_prompt})
    print(sql_response)

    # Try to extract JSON from ```json ... ``` block
    match = re.search(r"```json\s*(\{.*?\})\s*```", sql_response, re.DOTALL)
    if match:
        json_str = match.group(1)
    else:
        # Fallback: try to find the first {...}
        match2 = re.search(r"(\{.*\})", sql_response, re.DOTALL)
        if not match2:
            raise ValueError("No JSON object found in model response.")
        json_str = match2.group(1)

    data = json.loads(json_str)
    sql_query = data.get("SQL")
    if not sql_query:
        raise ValueError("No 'SQL' field found in JSON.")

    print("\n--- GENERATED SQL ---\n", sql_query)
    return sql_query

# ================== SQL EXECUTION ==================
def run_sql(query: str):
    try:
        with sqlite3.connect("example.db", timeout=10) as conn:
            cursor = conn.cursor()
            cursor.execute(query)

            # If it's a SELECT, fetch results
            if query.strip().lower().startswith("select"):
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]

                print("\n--- QUERY RESULTS ---")
                print("Columns:", columns)
                for row in rows:
                    print(row)
            else:
                # For INSERT/UPDATE/DELETE
                conn.commit()
                print("\n✅ Query executed successfully (non-SELECT).")
    except Exception as e:
        print("\n❌ Error executing SQL:", e)

# ================== RUN EVERYTHING ==================
# 1) Setup DB (run once per runtime)
setupdb()

# 2) Ask a natural language question
question = "give the data bace of id == 1"

# 3) Generate SQL using Gemini
sql_query = generate_sql(question)

# 4) Execute the generated SQL on SQLite
run_sql(sql_query)


ℹ️ Data already exists, skipping insert.

--- LLM RAW RESPONSE ---

```json
{
  "SQL": "SELECT * FROM customers WHERE id = 1"
}
```

--- GENERATED SQL ---
 SELECT * FROM customers WHERE id = 1

--- QUERY RESULTS ---
Columns: ['id', 'name', 'email', 'signup_date']
(1, 'Alice Johnson', 'alice@example.com', '2025-06-15')


RAg pipe line embeding and storage


In [9]:
 !pip install chromadb sentence-transformers  langchain-google-genai==2.0.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 7.2 MB/s eta 0:0